In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
from pettingzoo.classic import connect_four_v3
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"
from IPython.display import clear_output
import random

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-iqk44qm9 because the default path (/home/epic_euclid/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [19]:
#Dimensions du tableau
height=6
width=7

#Fonction qui initialise la matrice Q
def initialize_Q(): #Initialise la matrice Q (que des zéros) ####################################
    return np.zeros((height, width, height, width, width)) 
def find_best_legal(arr, mask):
    legal_gains=arr*mask
    best_action=np.argmax(legal_gains)
    if legal_gains[best_action] != 0:
        return best_action
    else :
        action=random.randint(0,6)
        while mask[action]==0:
            action=random.randint(0,6)
        return action
def left_legal(actions,mask):
    for i in range(len(actions)):
        if not (mask[i]==0) :
            return i
        
class Player:
    def __init__(self, _id, Q, epsilon=0.7, rng=None):  ############################
        self.name = 'Player '+str(_id)
        self.Q=initialize_Q()
        self.st=[None,None,None,None] #Etat
        self.last_st=[None,None,None,None] 
        self.last_action=None
        self._id=_id #Identifiant
        self.epsilon=epsilon
        self.left=False
        
    def get_action(self, obs, epsilon=None):
        mask=obs["action_mask"]
        if self.left:
            actions=[0,1,2,3,4,5,6]
            return left_legal(actions,mask)
        (a,b,c,d)=self.st
        if (random.uniform(0,1)<epsilon) or ((a,b) == (None,None)): #Exploration ou premier coupde la partie
            action=random.randint(0,6)
            while mask[action]==0:
                action=random.randint(0,6)
        else:
            (x1, y1, x2, y2)=self.st
            Q=self.Q
            #print(x,y)
            action=find_best_legal(Q[x1, y1, x2, y2], mask) #On choisit la meilleure action parmi celles qu'il peut jouer (meilleure dans le sens maximise Q)
        return action
    
    def highest_not_null(self, action, obs, terminated): #Fonction auxiliaire à la fonction pour déterminer l'état
        if not terminated:
            idx=1
        else:
            idx=0
        for i in range(height):
            if obs["observation"][:, action, idx][i] != 0: 
                return i

    def update_state(self,position):
        self.st[0]=self.st[2]
        self.st[1]=self.st[3]
        self.st[2]=position[0]
        self.st[3]=position[1]
        
    def update_Q(self, obs, action, alpha, gamma,show=False): #fonction qui update Q selon 
        next_action=self.get_action(obs, epsilon=0.0)
        [a,b,c,d]=self.st.copy()
        #print("state : ",a,b,c,d)
        [x,y,z,t]=self.last_st.copy()
        #print("last state : ",x,y,z,t)
        Q=self.Q
        pic=Q[x,y,z,t].copy() #Garder en mémoire les anciennes récompenses pour chaque action de l'état (x,y) (pour la visualisation)
        self.Q[x,y,z,t][action]=(1-alpha)*Q[x,y,z,t][action] + alpha*gamma*Q[a,b,c,d][next_action]#-Q[x,y][action]) #modif update_Q
        if show:
            print("Pour l'état ( "+str(x)+" , "+str(y)+" , "+str(z)+" , "+str(t)+" ), voici les anciennes récompenses : "+str(pic)+" et voici les nouvelles : "+str(self.Q[x,y,z,t]))
    def update_Q_reward(self, obs,r, action, alpha, gamma,show=False): #fonction qui update Q selon 
        [x,y,z,t]=self.st.copy()
        Q=self.Q
        pic=Q[x,y,z,t].copy() #Garder en mémoire les anciennes récompenses pour chaque action de l'état (x,y) (pour la visualisation)
        self.Q[x,y,z,t][action]=(1-alpha)*Q[x,y,z,t][action] + alpha*r 
        if show:
            print("Pour l'état ( "+str(x)+" , "+str(y)+" , "+str(z)+" , "+str(t)+" ), voici les anciennes récompenses : "+str(pic)+" et voici les nouvelles : "+str(self.Q[x,y,z,t]))
    
    def reset_states(self):
        self.st=[None,None,None,None]
        self.last_st=[None,None,None,None]

In [22]:
def play_game(env, agent0, agent1, alpha, gamma, display=False): #e epsilon coeff d'exploration
    done = False
    env.reset()
    obs, _, _, _, _ = env.last()
    l=[agent0, agent1]
    random.shuffle(l)
    new_state=(None,None,None,None)
    while not done:
        for i, agent in enumerate(l):
            position=(new_state[2],new_state[3]) #Position du dernier coup de l'adversaire
            agent.update_state(position) #Etat de l'agent mis à jour avec ce qu'a joué l'adversaire
            last_state= agent.st.copy() #Récupération de l'état de l'agent
            [a,b,c,d]=agent.last_st.copy()
            if (a!=None) and (b!=None) :#and (c!=None) and (d !=None): #Mise à jour des récompenses dans Q
                agent.update_Q(obs, agent.last_action, alpha, gamma, show=False) ############ A Regarder
            
            #Trouver l'action
            e=agent.epsilon
            action = agent.get_action(obs, epsilon=e) 
            agent.last_action=action
            env.step(action) #Préciser à l'environnement pettingZoo que l'action prise
            obs, reward, terminated, _, _ = env.last()
            
            if reward==1:
                agent.update_Q_reward(obs,reward, action, alpha, gamma,show=False)
            position=(agent.highest_not_null(action, obs, terminated),action) #Position du coup de l'agent

            if display:
                clear_output(wait=True)
                plt.imshow(env.render())
                plt.show()
            done = terminated #Déroulement fini
            
            if np.sum(obs["action_mask"]) == 0: #Vérification d'une éventuelle égalité
                if display: 
                    print('Draw')
                done=True
                break
                
            agent.last_st=last_state
            agent.update_state(position) #Mise à jour de l'état de l'agent (l'état est intermédiaire avant l'udating en début de boucle)
            new_state=agent.st.copy()
            if done: #Victoire d'un des joueurs
                if display:
                    print(f"{agent.name} : won")
                    print(obs['observation'][:, :, 0]- obs['observation'][:, :, 1])
                    print('last_state : ', last_state)
                    print('action : ', action)
                break       

In [23]:
def play_game_count(env, agent0, agent1, alpha, gamma, display=False, dic={}): #e epsilon coeff d'exploration
    done = False
    env.reset()
    obs, _, _, _, _ = env.last()
    l=[agent0, agent1]
    random.shuffle(l)
    new_state=(None,None,None,None)
    while not done:
        #print("")
        #print("")
        for i, agent in enumerate(l):
            position=(new_state[2],new_state[3]) #Position du dernier coup de l'adversaire
            agent.update_state(position) #Etat de l'agent mis à jour avec ce qu'a joué l'adversaire
            last_state= agent.st.copy() #Récupération de l'état de l'agent
            [a,b,c,d]=agent.last_st.copy()
            if (a!=None) and (b!=None) :#and (c!=None) and (d !=None): #Mise à jour des récompenses dans Q
                agent.update_Q(obs, agent.last_action, alpha, gamma, show=False) ############ A Regarder
            
            #Trouver l'action
            e=agent.epsilon
            action = agent.get_action(obs, epsilon=e) 
            agent.last_action=action
            env.step(action) #Préciser à l'environnement pettingZoo que l'action prise
            obs, reward, terminated, _, _ = env.last()
            if reward==1:
                agent.update_Q_reward(obs,reward, action, alpha, gamma,show=False)
            position=(agent.highest_not_null(action, obs, terminated),action) #Position du coup de l'agent

            if display:
                clear_output(wait=True)
                plt.imshow(env.render())
                plt.show()
            done = terminated #Déroulement fini
            if np.sum(obs["action_mask"]) == 0: #Vérification d'une éventuelle égalité
                if display: 
                    print('Draw')
                done=True
                result='Draw'
                break
                
            agent.last_st=last_state
            agent.update_state(position) #Mise à jour de l'état de l'agent (l'état est intermédiaire avant l'udating en début de boucle)
            new_state=agent.st.copy()
            if done: #Victoire d'un des joueurs
                if display:
                    print(f"{agent.name} : won")
                    print(obs['observation'][:, :, 0]- obs['observation'][:, :, 1])
                    print('last_state : ', last_state)
                    print('action : ', action)
                done=True
                result=agent.name
                break       
    dic[result]+=1

In [24]:
def multiple_games_count(env, agent0, agent1, alpha, gamma , count):
    dic={}
    dic['Draw']=0
    dic[agent0.name]=0
    dic[agent1.name]=0
    for i in tqdm(range(count)):
        agent0.reset_states()
        agent1.reset_states()
        play_game_count(env, agent0, agent1, alpha, gamma, display=False, dic=dic)
    return dic

Entraînement et évaluation

In [18]:
env = connect_four_v3.env(render_mode="rgb_array")
env.reset()
agent0=Player(_id=1, Q=initialize_Q(), epsilon=0.3) #Il explore 30% du tps
random_player=Player(_id=2, Q=initialize_Q(), epsilon =1) #Random explore 100% du tps
epsilon_training_mate=1
training_mate=Player(_id=2, Q=initialize_Q(), epsilon =epsilon_training_mate)
alpha=0.1
gamma=0.9
loop=10000

In [16]:
#training_mate.left=True
for i in tqdm(range(9*loop)): #Entraînement
    agent0.reset_states()
    training_mate.reset_states()
    play_game(env, agent0, training_mate, alpha, gamma , display=False)

100%|██████████| 90000/90000 [04:29<00:00, 334.09it/s]


In [17]:
agent0.epsilon=0 #On le passe en mode exploitation
random_player.left=True
multiple_games_count(env, agent0, random_player, alpha, gamma , 4*loop)    

100%|██████████| 40000/40000 [00:49<00:00, 800.98it/s]


{'Draw': 0, 'Player 1': 14204, 'Player 2': 25796}

Parameters Choice

In [7]:
dic_={}
for alpha in [0.001, 0.01, 0.1]:
    for gamma in [0.5,0.9]:
        print("Processing : ",alpha,gamma)
        env = connect_four_v3.env(render_mode="rgb_array")
        env.reset()
        agent0=Player(_id=1, Q=initialize_Q(), epsilon=0.3)
        training_mate=Player(_id=2, Q=initialize_Q(), epsilon=1)
        random_player=Player(_id="Random", Q=initialize_Q(), epsilon=1)
        e=0.0
        loop=40000
        for i in tqdm(range(loop)): #training
            agent0.reset_states()
            training_mate.reset_states()
            play_game(env, agent0, training_mate, alpha, gamma , display=False)
        dic=multiple_games_count(env, agent0, random_player, alpha, gamma , loop) 
        dic_[(alpha,gamma)]=dic
dic_

Processing :  0.001 0.5


100%|██████████| 40000/40000 [02:23<00:00, 278.88it/s]


Processing :  0.001 0.9


100%|██████████| 40000/40000 [02:28<00:00, 269.80it/s]


Processing :  0.01 0.5


...

Processing :  0.1 0.5


100%|██████████| 40000/40000 [01:54<00:00, 349.96it/s]


Processing :  0.1 0.9


100%|██████████| 40000/40000 [01:57<00:00, 341.19it/s]


{(0.001, 0.5): {'Draw': 83, 'Player 1': 25188, 'Player Random': 14729},
 (0.001, 0.9): {'Draw': 82, 'Player 1': 23988, 'Player Random': 15930},
 (0.01, 0.5): {'Draw': 57, 'Player 1': 25239, 'Player Random': 14704},
 (0.01, 0.9): {'Draw': 76, 'Player 1': 24632, 'Player Random': 15292},
 (0.1, 0.5): {'Draw': 26, 'Player 1': 28675, 'Player Random': 11299},
 (0.1, 0.9): {'Draw': 40, 'Player 1': 28845, 'Player Random': 11115}}